<a href="https://colab.research.google.com/github/G1tGusta/Atividades-Faculdade/blob/main/TriagemDeEnfermagemComFilas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import heapq
from typing import Dict, List, Tuple, Any

# ===================== #
# 1. CONSTANTES GERAIS  #
# ===================== #

COR_PRIORIDADE: Dict[str, int] = {
    "vermelho": 4,
    "laranja": 3,
    "amarelo": 2,
    "azul": 1
}

TEMPO_CONSULTA_MINUTOS = 20
ESPECIALIDADES = [
    "Clínica Médica",
    "Neurologia",
    "Ortopedia",
    "Dermatologia",
    "Pediatria",
    "Cardiologia"
]

# ============================= #
# 2. FUNÇÕES DE TEMPO #
# ============================= #

def hora_para_minutos(hora: str) -> int:
    h, m = map(int, hora.split(":"))
    return h * 60 + m

def minutos_para_hora(mins: int) -> str:
    return f"{mins // 60:02d}:{mins % 60:02d}"

# ===================== #
# 3. CLASSE PACIENTE    #
# ===================== #

class Paciente:
    def __init__(self, nome: str, idade: int, cor: str, especialidade: str, hora_chegada: str):
        if cor.lower() not in COR_PRIORIDADE:
            raise ValueError("Cor de urgência inválida.")
        self.nome = nome
        self.idade = idade
        self.cor = cor.lower()
        self.especialidade = especialidade
        self.hora_chegada = hora_para_minutos(hora_chegada)
        self.prioridade = COR_PRIORIDADE[self.cor]

# ============================== #
# 4. CLASSE TRIAGEM #
# ============================== #

class TriagemHospitalar:
    def __init__(self):
        self.filas: Dict[str, List[Tuple[Tuple[int, int, int], Paciente]]] = {esp: [] for esp in ESPECIALIDADES}
        self.ultima_consulta: Dict[str, int] = {}
        self.contador_global = 0

    def adicionar_paciente(self, paciente: Paciente):
        self.contador_global += 1
        chave = (-paciente.prioridade, paciente.hora_chegada, self.contador_global)
        heapq.heappush(self.filas[paciente.especialidade], (chave, paciente))
        if paciente.especialidade not in self.ultima_consulta:
            self.ultima_consulta[paciente.especialidade] = paciente.hora_chegada

    def proximo(self, especialidade: str):
        fila = self.filas.get(especialidade)
        if not fila:
            raise IndexError(f"Não há pacientes aguardando em {especialidade}.")
        _, paciente = heapq.heappop(fila)
        inicio = max(paciente.hora_chegada, self.ultima_consulta.get(especialidade, paciente.hora_chegada))
        fim = inicio + TEMPO_CONSULTA_MINUTOS
        self.ultima_consulta[especialidade] = fim
        return paciente, minutos_para_hora(inicio), minutos_para_hora(fim)

    def previsao(self, especialidade: str) -> List[Dict[str, Any]]:
        fila_copia = list(self.filas.get(especialidade, []))
        if not fila_copia:
            return []

        heapq.heapify(fila_copia)
        tempo_livre = self.ultima_consulta.get(especialidade, 0)
        previsao = []

        while fila_copia:
            _, p = heapq.heappop(fila_copia)
            inicio = max(p.hora_chegada, tempo_livre)
            fim = inicio + TEMPO_CONSULTA_MINUTOS
            espera = inicio - p.hora_chegada
            previsao.append({
                "nome": p.nome,
                "urgencia": p.cor.capitalize(),
                "chegada": minutos_para_hora(p.hora_chegada),
                "inicio": minutos_para_hora(inicio),
                "fim": minutos_para_hora(fim),
                "espera": espera
            })
            tempo_livre = fim
        return previsao

    def tempo_medio_espera(self, especialidade: str) -> float:
        previsoes = self.previsao(especialidade)
        if not previsoes:
            return 0.0
        return round(sum(p["espera"] for p in previsoes) / len(previsoes), 2)

# ===================== #
# 5. EXECUÇÃO PRINCIPAL #
# ===================== #

if __name__ == "__main__":
    hospital = TriagemHospitalar()

    print("\nSISTEMA DE TRIAGEM HOSPITALAR")
    print("=" * 70)

    # ---------- Cadastro ----------
    pacientes = [
        # Clínica Médica
        ("João Ribeiro", 65, "laranja", "Clínica Médica", "07:50"),
        ("Camila Duarte", 42, "amarelo", "Clínica Médica", "07:55"),
        ("Fernanda Alves", 58, "azul", "Clínica Médica", "08:10"),
        ("Tiago Santos", 37, "vermelho", "Clínica Médica", "07:52"),

        # Neurologia
        ("Sofia Mendes", 32, "vermelho", "Neurologia", "07:58"),
        ("Henrique Moraes", 41, "amarelo", "Neurologia", "08:00"),
        ("Juliana Costa", 29, "azul", "Neurologia", "08:05"),
        ("Gustavo Lima", 53, "laranja", "Neurologia", "08:03"),

        # Ortopedia
        ("Lúcia Torres", 70, "vermelho", "Ortopedia", "08:04"),
        ("Carlos Souza", 60, "amarelo", "Ortopedia", "08:06"),
        ("Paula Nunes", 28, "laranja", "Ortopedia", "08:08"),
        ("Rodrigo Farias", 33, "azul", "Ortopedia", "08:12"),
        ("André Silva", 49, "amarelo", "Ortopedia", "08:10"),

        # Dermatologia
        ("Marina Rocha", 26, "laranja", "Dermatologia", "08:02"),
        ("Alice Monteiro", 44, "amarelo", "Dermatologia", "08:05"),
        ("Leonardo Prado", 36, "azul", "Dermatologia", "08:15"),

        # Pediatria
        ("Pedro Costa", 12, "azul", "Pediatria", "08:10"),
        ("Laura Ferreira", 7, "vermelho", "Pediatria", "08:07"),
        ("Isabela Ramos", 4, "amarelo", "Pediatria", "08:12"),

        # Cardiologia
        ("Lucas Pereira", 45, "amarelo", "Cardiologia", "07:50"),
        ("Beatriz Lima", 51, "laranja", "Cardiologia", "08:08"),
        ("Eduardo Souza", 63, "vermelho", "Cardiologia", "08:05"),
        ("Paulo Ricardo", 57, "azul", "Cardiologia", "08:20")
    ]

    for nome, idade, cor, esp, hora in pacientes:
        hospital.adicionar_paciente(Paciente(nome, idade, cor, esp, hora))

    print("\nPacientes cadastrados com sucesso.\n")

    # ---------- Mostrar previsão por especialidade ----------
    for esp in ESPECIALIDADES:
        previsao = hospital.previsao(esp)
        if not previsao:
            continue

        print(f"\n{esp.upper()}")
        print("-" * 80)
        print(f"{'Nome':20} | {'Urgência':10} | {'Chegada':8} | {'Início':8} | {'Fim':8} | {'Espera (min)':12}")
        print("-" * 80)
        for p in previsao:
            print(f"{p['nome'][:20]:20} | {p['urgencia']:10} | {p['chegada']:8} | {p['inicio']:8} | {p['fim']:8} | {p['espera']:>12}")
        print("-" * 80)
        print(f"Tempo médio de espera: {hospital.tempo_medio_espera(esp)} minutos")

    # ---------- Chamada de alguns pacientes ----------
    print("\nINÍCIO DO ATENDIMENTO - CHAMANDO PRIMEIROS PACIENTES")
    print("=" * 70)
    for esp in ["Ortopedia", "Cardiologia", "Pediatria"]:
        try:
            paciente, inicio, fim = hospital.proximo(esp)
            print(f"{esp}: {paciente.nome} ({paciente.cor.capitalize()}) -> Início {inicio}, Fim {fim}")
        except IndexError:
            print(f"{esp}: sem pacientes na fila.")

    # ---------- Resumo final ----------
    print("\nTEMPO MÉDIO DE ESPERA - RESUMO GERAL")
    print("=" * 70)
    for esp in ESPECIALIDADES:
        media = hospital.tempo_medio_espera(esp)
        print(f"{esp:25} -> {media:5.2f} min")
    print("=" * 70)
    print("Simulação encerrada.\n")



SISTEMA DE TRIAGEM HOSPITALAR

Pacientes cadastrados com sucesso.


CLÍNICA MÉDICA
--------------------------------------------------------------------------------
Nome                 | Urgência   | Chegada  | Início   | Fim      | Espera (min)
--------------------------------------------------------------------------------
Tiago Santos         | Vermelho   | 07:52    | 07:52    | 08:12    |            0
João Ribeiro         | Laranja    | 07:50    | 08:12    | 08:32    |           22
Camila Duarte        | Amarelo    | 07:55    | 08:32    | 08:52    |           37
Fernanda Alves       | Azul       | 08:10    | 08:52    | 09:12    |           42
--------------------------------------------------------------------------------
Tempo médio de espera: 25.25 minutos

NEUROLOGIA
--------------------------------------------------------------------------------
Nome                 | Urgência   | Chegada  | Início   | Fim      | Espera (min)
---------------------------------------------------